In [3]:
!pip install datasets
!pip install trl

  Using cached trl-0.9.4-py3-none-any.whl (226 kB)


In [5]:
!pip install sentencepiece

     |████████████████████████████████| 1.3 MB 5.1 MB/s eta 0:00:01


In [6]:
!pip install peft

     |████████████████████████████████| 251 kB 5.1 MB/s eta 0:00:01
  Using cached accelerate-0.31.0-py3-none-any.whl (309 kB)
  Using cached torch-2.3.1-cp38-cp38-manylinux1_x86_64.whl (779.1 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached triton-2.3.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (168.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)


In [7]:
!pip install tensorboardX

     |████████████████████████████████| 101 kB 2.6 MB/s ta 0:00:011
     |████████████████████████████████| 309 kB 40.8 MB/s eta 0:00:01


In [1]:
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, AutoModelForSeq2SeqLMWithValueHead
from datasets import load_dataset
from transformers import AutoTokenizer
import pandas as pd

### Configuration

In [2]:
ppo_config = PPOConfig(
    model_name="SFT_Model_T5-Small",
    learning_rate=1.41e-5,
    # log_with="wandb",
)

In [3]:
sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": 1}

### Load Dataset

In [4]:
ppo_dataset = load_dataset("gen_sft_dataset.py", trust_remote_code=True) # gen_ppo_dataset
ppo_dataset

DatasetDict({
    train: Dataset({
        features: ['weibo', 'resp'],
        num_rows: 25140
    })
    validation: Dataset({
        features: ['weibo', 'resp'],
        num_rows: 8670
    })
})

In [5]:
ppo_dataset["train"]["resp"][1]

'不用呼吸机光速下班'

### Load Pre-trained SFT Model

In [7]:
# model = AutoModelForCausalLMWithValueHead.from_pretrained("./SFT_Model_T5-Small")
# ref_model = AutoModelForCausalLMWithValueHead.from_pretrained("./SFT_Model_T5-Small")
# tokenizer = AutoTokenizer.from_pretrained("./SFT_Model_T5-Small")

model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained("./SFT_Model_T5-Small/checkpoint-15720")
ref_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained("./SFT_Model_T5-Small/checkpoint-15720")
tokenizer = AutoTokenizer.from_pretrained("./SFT_Model_T5-Small/checkpoint-15720")

tokenizer.pad_token = tokenizer.eos_token

# from transformers import AutoModelForSeq2SeqLM, PreTrainedModel

# base_model = PreTrainedModel.from_pretrained("./SFT_Model_ChatGLM/checkpoint-4000", trust_remote_code = True)
# # base_model = PreTrainedModelWrapper.from_pretrained(base_model)
# model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(base_model)
# ref_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(base_model)
# tokenizer = AutoTokenizer.from_pretrained("./SFT_Model_ChatGLM/checkpoint-4000")

# tokenizer.pad_token = tokenizer.eos_token

In [8]:
def tokenize(sample):
    # sample["input_ids"] = tokenizer.encode(sample["weibo"])
    sample = tokenizer(sample["weibo"])
    sample["query"] = tokenizer.decode(sample["input_ids"])
    return sample

ppo_train_dataset = ppo_dataset["train"]
ppo_train_dataset = ppo_train_dataset.map(tokenize) # Bug Fixed (Not In-place)
ppo_train_dataset.set_format(type="torch") # Important! Turn into tensors 

In [9]:
ppo_train_dataset

Dataset({
    features: ['weibo', 'resp', 'input_ids', 'attention_mask', 'query'],
    num_rows: 25140
})

In [10]:
print(ppo_train_dataset[0]['input_ids'])

tensor([ 1713,   518,  9697,     2,  4663,     3,     2,   536, 13572,     2,
        22773,     2,  2596,  3486,     6,  2596,  4278,     6,  2596,  6832,
            1])


### Initialize PPOTrainer 

In [11]:
# from transformers import DataCollatorForSeq2Seq

# data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

In [12]:
ppo_trainer = PPOTrainer(ppo_config, model, ref_model, tokenizer, dataset=ppo_train_dataset, data_collator=collator)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


### Load Tuned Reward Model Trained in RM Phase

In [13]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

# rm_model = AutoModelForSequenceClassification.from_pretrained("./RM_model/checkpoint-1560")
# rm_tokenizer = AutoTokenizer.from_pretrained("./RM_model/checkpoint-1560")

rm_model = AutoModelForSequenceClassification.from_pretrained("./RM_model7/checkpoint-11700")
rm_tokenizer = AutoTokenizer.from_pretrained("./RM_model7/checkpoint-11700")
rm_tokenizer.pad_token = tokenizer.eos_token
# GPT
rm_tokenizer.padding_side = "right"  # This ensures consistent padding
rm_model.config.pad_token_id = rm_tokenizer.pad_token_id

sentiment_pipe = pipeline("sentiment-analysis", model=rm_model, tokenizer=rm_tokenizer, padding=True, truncation=True, max_length=32)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


### Training Loop for Model Optimization

In [15]:
from tqdm import tqdm

In [16]:
ppo_trainer.dataloader

In [17]:
print(ppo_train_dataset[:5])

{'weibo': ['#WTT冠军赛布达佩斯站#\xa0男单1/4决赛林高远3-0宇田幸矢11-3，11-4，11-7', '#WTT冠军赛布达佩斯站#\xa0男单1/4决赛林高远3-0宇田幸矢11-3，11-4，11-7', '#WTT冠军赛布达佩斯站#\xa0男单1/4决赛林高远3-0宇田幸矢11-3，11-4，11-7', '#WTT冠军赛布达佩斯站#\xa0男单1/4决赛林高远3-0宇田幸矢11-3，11-4，11-7', '#WTT冠军赛布达佩斯站#\xa0男单1/4决赛林高远3-0宇田幸矢11-3，11-4，11-7'], 'resp': ['别把我帅死林高远一直这么坚定下去吧！！！！别有太大压力，战胜自己就够了！！！！我永远相信小林将军', '不用呼吸机光速下班', '“人家肯定拼你的”，“对，所以说这个是被拼的时候，自己要扛住压力这个事。”今天发球落点速度节奏变化都好棒，让大家欣赏一下速度流明天的半决赛将迎战本站第一个右手，放平心态，你谁都不怕！', '“坚定都溢出来了”“怎么可以防的如此自如啊”“非常自信/冷静啊”…没有不可治愈的伤痛，没有不可结束的沉沦。所有失去的，会以另一种方式归来。林高远，好样的，这场打的真漂亮，继续加油，愈战愈勇！', '“全方位的碾压让对方看不到一丝希望！”林高远牛逼！是意气风发的小林将军林高远继续加油！！！！'], 'input_ids': tensor([[ 1713,   518,  9697,     2,  4663,     3,     2,   536, 13572,     2,
         22773,     2,  2596,  3486,     6,  2596,  4278,     6,  2596,  6832,
             1],
        [ 1713,   518,  9697,     2,  4663,     3,     2,   536, 13572,     2,
         22773,     2,  2596,  3486,     6,  2596,  4278,     6,  2596,  6832,
             1],
        [ 1713,   518,  9697,     2,  4663,   

In [18]:
import torch

In [19]:
# from trl.trainer import PPOTrainer

# class DebugPPOTrainer(PPOTrainer):
#     def compute_rewards(self, scores, logprobs, ref_logprobs, masks):
#         print(f"Mask: {masks}")  # Print the mask tensor
#         return super().compute_rewards(scores, logprobs, ref_logprobs, masks)

In [20]:
# Generation Settings
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 0.7,
    "temperature": 0.95, # This one is set implicitly in SFTTrainer as well.
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}


for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)): # Need the specific type of data_collator
    print(f"\nStart Epoch {epoch}")
    # print(batch)
    query_tensors = batch["input_ids"]
    attention_masks = batch["attention_mask"]
    # attention_masks = [query_tensors != tokenizer.pad_token_id]
    # print(attention_masks)

    #### Get response from gpt2
    response_tensors = []
    for i, query in enumerate(query_tensors):
        # gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = 32
        # print(attention_masks[i])
        response = ppo_trainer.generate(query, **generation_kwargs, attention_mask=attention_masks[i].unsqueeze(0))
        response_tensors.append(response.squeeze()[-32:])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    #### Compute sentiment score
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]
    print(f"Rewards: {rewards}")

    #### Run PPO step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

0it [00:00, ?it/s]


Start Epoch 0


/usr/local/lib/python3.8/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Rewards: [tensor(-0.3165), tensor(-0.0251), tensor(-1.0098), tensor(-0.1059), tensor(-0.9854), tensor(-0.6910), tensor(-0.6012), tensor(-0.4122), tensor(-0.8500), tensor(-0.5225), tensor(-0.8987), tensor(-1.0042), tensor(-0.6118), tensor(-0.3549), tensor(-0.2822), tensor(-0.7444), tensor(-1.4790), tensor(-1.2325), tensor(-0.6816), tensor(-0.1488), tensor(0.3995), tensor(-1.4229), tensor(-0.2256), tensor(0.1444), tensor(-0.3305), tensor(-0.3921), tensor(-0.4582), tensor(-0.5637), tensor(-0.9589), tensor(-0.1358), tensor(-1.1236), tensor(-0.9890), tensor(-0.4730), tensor(0.6373), tensor(-1.1058), tensor(-0.9395), tensor(-0.0384), tensor(0.1405), tensor(-0.6538), tensor(-1.0177), tensor(-0.3271), tensor(-0.9890), tensor(-0.6431), tensor(-0.3343), tensor(-0.2904), tensor(-1.2058), tensor(0.0029), tensor(-1.1483), tensor(-0.2259), tensor(0.2283), tensor(-0.9854), tensor(0.0175), tensor(-0.5428), tensor(0.1127), tensor(-1.1798), tensor(-0.0141), tensor(-1.2325), tensor(-0.8022), tensor(-1.03

1it [00:46, 46.10s/it]

...

194it [2:34:49, 46.42s/it]


Start Epoch 194
Rewards: [tensor(0.0623), tensor(-1.1262), tensor(-0.6160), tensor(-0.7769), tensor(-1.1229), tensor(-0.9729), tensor(-0.2174), tensor(-0.5694), tensor(-0.4864), tensor(-0.2455), tensor(-0.1637), tensor(0.0793), tensor(-0.1302), tensor(-0.7005), tensor(-0.5445), tensor(-0.4100), tensor(-0.6716), tensor(-0.8113), tensor(-0.3660), tensor(-1.0460), tensor(-0.3665), tensor(-0.5774), tensor(-0.6987), tensor(-1.0256), tensor(-0.9020), tensor(-0.8869), tensor(-0.0251), tensor(0.1730), tensor(-0.4501), tensor(-0.3240), tensor(-0.6728), tensor(-0.7018), tensor(-0.6449), tensor(-0.2109), tensor(0.1077), tensor(-0.6863), tensor(-0.1663), tensor(-0.6635), tensor(-0.7347), tensor(0.2108), tensor(-0.8831), tensor(-0.5368), tensor(-0.5553), tensor(-0.9968), tensor(-1.0729), tensor(-0.8253), tensor(0.0012), tensor(-1.0519), tensor(0.0332), tensor(-0.0853), tensor(-0.0905), tensor(-0.5514), tensor(0.1405), tensor(-1.1427), tensor(-0.6050), tensor(-0.6201), tensor(-0.0218), tensor(-0.78

195it [2:35:35, 46.18s/it]


Start Epoch 195
Rewards: [tensor(-0.1269), tensor(-0.0121), tensor(-0.0526), tensor(-0.6910), tensor(-0.2308), tensor(-0.5611), tensor(-0.5726), tensor(-0.8334), tensor(-0.1059), tensor(-0.8386), tensor(-0.8307), tensor(-0.1133), tensor(-0.8831), tensor(-0.2357), tensor(-1.1749), tensor(-1.2142), tensor(-0.0428), tensor(0.3988), tensor(0.2955), tensor(-0.6385), tensor(-0.6779), tensor(-0.4140), tensor(-1.2181), tensor(-0.5049), tensor(-0.4921), tensor(-0.1657), tensor(-0.6346), tensor(-0.9787), tensor(-0.5578), tensor(-1.2466), tensor(-0.7488), tensor(-0.4016), tensor(-0.5823), tensor(-0.2614), tensor(-0.4304), tensor(-0.5480), tensor(-0.6987), tensor(-0.2962), tensor(-0.8917), tensor(-0.1214), tensor(-0.0474), tensor(-0.2114), tensor(-0.9052), tensor(0.0132), tensor(-0.0950), tensor(-0.0633), tensor(-0.5551), tensor(-0.2884), tensor(-0.3264), tensor(-0.9150), tensor(-1.1306), tensor(-0.5309), tensor(-0.3875), tensor(-0.4782), tensor(0.0158), tensor(-0.8307), tensor(-0.1913), tensor(-

196it [2:36:20, 47.86s/it]


In [22]:
device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug

In [28]:
#### get a batch from the dataset
bs = 16
game_data = dict()
ppo_train_dataset.set_format("pandas")
df_batch = ppo_train_dataset[:].sample(bs)
game_data["query"] = df_batch["query"].tolist()
query_tensors = df_batch["input_ids"].tolist()

response_tensors_ref, response_tensors = [], []

#### get response from gpt2 and gpt2_ref
for i in range(bs):
    # gen_len = output_length_sampler()
    generation_kwargs["max_new_tokens"] = 32
    output = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), **generation_kwargs
    ).squeeze()[-32:]
    response_tensors_ref.append(output)
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), **generation_kwargs
    ).squeeze()[-32:]
    response_tensors.append(output)

#### decode responses
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data["response (after)"] = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

#### sentiment analysis of query/response pairs before/after
texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results

,query,response (before),response (after),rewards (before),rewards (after)
0,"#<unk>##<unk>#<unk>,<unk>,<unk>,<unk>,<unk>,<u...",<pad> <unk></s>,"<pad> <unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk...",-0.820149,-0.820149
1,"<unk>?#<unk>#<unk>,<unk>,<unk>,<unk>“<unk>”!#<...",<pad> <unk></s>,"<pad> <unk>,<unk>,<unk></s>",-0.113685,-0.113685
2,"<unk>,<unk>..L<unk>#<unk>##<unk>#</s>",<pad> <unk></s>,"<pad> <unk>,<unk>?</s>",-0.961194,-0.770309
3,"#<unk>WTT<unk>#<unk>!<unk>,WTT<unk>, #<unk>#,<...",<pad> <unk>!!!<unk></s>,<pad> <unk> <unk> <unk> <unk> <unk> <unk></s>,-1.202836,-1.202836
4,"<unk>,<unk>#<unk>#L<unk>bot</s>",<pad> <unk></s>,"<pad> <unk>,<unk></s>",-1.191703,-1.107892
5,#<unk>900<unk>##<unk>900<unk>#:<unk>6<unk>25<u...,"<pad> <unk>,<unk>,<unk>,<unk></s>","<pad> <unk>,900<unk> <unk></s>",-0.665839,-0.665839
6,"#<unk>#4<unk>8<unk>,#<unk>#<unk>,<unk>7<unk>,<...",<pad> <unk> <unk> <unk> <unk> <unk> <unk> <unk...,"<pad> <unk>,<unk>,<unk>,<unk>,<unk>,<unk></s>",-1.178802,-1.178802
7,#<unk>#<unk>:<unk>!#<unk>#<unk>......<unk>“<un...,"<pad> <unk>,<unk></s>","<pad> <unk>,<unk></s>",-0.162369,-0.162369
8,"#<unk>#<unk>2<unk>26<unk>,<unk>,<unk>,<unk>27<...","<pad> <unk>,<unk></s>",<pad> <unk></s>,-0.915403,-0.915403
9,"TTG<unk>#TTG<unk>eStar#<unk>,<unk>,<unk>,<unk>...",<pad> <unk></s>,"<pad> <unk>,<unk>,<unk>,<unk></s>",-0.624466,-0.624466


In [29]:
print("mean:")
display(df_results[["rewards (before)", "rewards (after)"]].mean())
print()
print("median:")
display(df_results[["rewards (before)", "rewards (after)"]].median())

mean:


rewards (before)   -0.668355
rewards (after)    -0.651187
dtype: float64


median:


rewards (before)   -0.660078
rewards (after)    -0.660078
dtype: float64

In [30]:
model.save_pretrained("ppo-tuned-t5-small")
tokenizer.save_pretrained("ppo-tuned-t5-small")

('ppo-tuned-t5-small/tokenizer_config.json',
 'ppo-tuned-t5-small/special_tokens_map.json',
 'ppo-tuned-t5-small/spiece.model',
 'ppo-tuned-t5-small/added_tokens.json',
 'ppo-tuned-t5-small/tokenizer.json')